In [1]:
!nvidia-smi

Tue Feb  8 02:12:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Instações

In [2]:
!pip install -q  transformers num2words pytorch_lightning neptune-client

     |████████████████████████████████| 3.5 MB 16.3 MB/s 
     |████████████████████████████████| 101 kB 15.0 MB/s 
     |████████████████████████████████| 527 kB 69.8 MB/s 
     |████████████████████████████████| 301 kB 72.0 MB/s 
     |████████████████████████████████| 895 kB 58.3 MB/s 
     |████████████████████████████████| 6.8 MB 47.7 MB/s 
     |████████████████████████████████| 67 kB 6.2 MB/s 
     |████████████████████████████████| 596 kB 67.6 MB/s 
     |████████████████████████████████| 397 kB 74.9 MB/s 
     |████████████████████████████████| 829 kB 67.1 MB/s 
     |████████████████████████████████| 133 kB 67.9 MB/s 
     |████████████████████████████████| 952 kB 65.6 MB/s 
     |████████████████████████████████| 1.1 MB 62.6 MB/s 
     |████████████████████████████████| 53 kB 2.5 MB/s 
     |████████████████████████████████| 180 kB 74.1 MB/s 
     |████████████████████████████████| 131 kB 72.7 MB/s 
     |████████████████████████████████| 79 kB 9.7 MB/s 
     |██████████████

In [3]:
# mound drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
# define paths

CODE_TRAIN_PATH = '/content/drive/MyDrive/MESTRADO/projeto/Experimentos/SCAN/Verificador/Verifier-T5/main-training_rerank.py'
%env CODE_TRAIN_PATH=$CODE_TRAIN_PATH


OUT_PATH = '/content/drive/MyDrive/MESTRADO/projeto/Experimentos/SCAN/Verificador/Verifier-T5/3e-4/'
%env OUT_PATH=$OUT_PATH

TRAIN_SAMPLES = '/content/drive/MyDrive/MESTRADO/projeto/Experimentos/SCAN/Verificador/generate-train-examples/train_hipoteses.tsv'
#TRAIN_SAMPLES = '/content/drive/MyDrive/MESTRADO/projeto/Experimentos/SCAN/Verificador/Verifier-T5/train_new.tsv'
%env TRAIN_SAMPLES=$TRAIN_SAMPLES

TEST_SAMPLES = '/content/drive/MyDrive/MESTRADO/projeto/Experimentos/SCAN/Verificador/generate-train-examples/test_hipoteses.tsv'
#TEST_SAMPLES = '/content/drive/MyDrive/MESTRADO/projeto/Experimentos/SCAN/Verificador/Verifier-T5/train_new.tsv'
%env TEST_SAMPLES=$TEST_SAMPLES

MODEL_RESTORE = '/content/drive/MyDrive/MESTRADO/projeto/Experimentos/SCAN/Verificador/Verifier-T5/3e-4/last.ckpt'
%env MODEL_RESTORE=$MODEL_RESTORE

LOG_RESTORE = '/content/drive/MyDrive/MESTRADO/projeto/Experimentos/SCAN/Verificador/Verifier-T5/3e-4/metrics_seed_27_test.tsv'
%env LOG_RESTORE=$LOG_RESTORE

CHECKPOINT_RESULT = '/content/drive/MyDrive/MESTRADO/projeto/Experimentos/SCAN/Verificador/Verifier-T5/top_k_4_num_return_sequences_100_t5_base_test_55_acc.json'
%env CHECKPOINT_RESULT=$CHECKPOINT_RESULT

env: CODE_TRAIN_PATH=/content/drive/MyDrive/MESTRADO/projeto/Experimentos/SCAN/Verificador/Verifier-T5/main-training_rerank.py
env: OUT_PATH=/content/drive/MyDrive/MESTRADO/projeto/Experimentos/SCAN/Verificador/Verifier-T5/3e-4/
env: TRAIN_SAMPLES=/content/drive/MyDrive/MESTRADO/projeto/Experimentos/SCAN/Verificador/generate-train-examples/train_hipoteses.tsv
env: TEST_SAMPLES=/content/drive/MyDrive/MESTRADO/projeto/Experimentos/SCAN/Verificador/generate-train-examples/test_hipoteses.tsv
env: MODEL_RESTORE=/content/drive/MyDrive/MESTRADO/projeto/Experimentos/SCAN/Verificador/Verifier-T5/3e-4/last.ckpt
env: LOG_RESTORE=/content/drive/MyDrive/MESTRADO/projeto/Experimentos/SCAN/Verificador/Verifier-T5/3e-4/metrics_seed_27_test.tsv
env: CHECKPOINT_RESULT=/content/drive/MyDrive/MESTRADO/projeto/Experimentos/SCAN/Verificador/Verifier-T5/top_k_4_num_return_sequences_100_t5_base_test_55_acc.json


### Treinamento

In [ ]:
#  --restore_checkpoint=$MODEL_RESTORE\
#  --restore_checkpoint=$MODEL_RESTORE \
#  --restore_logging=$LOG_RESTORE \
#  --test_checkpoint='True'\
!python -u $CODE_TRAIN_PATH \
  --output_dir=$OUT_PATH \
  --model_name_or_path=t5-base\
  --result_checkpoint=$CHECKPOINT_RESULT\
  --input_train=$TRAIN_SAMPLES \
  --input_val=$TEST_SAMPLES \
  --input_test=$TEST_SAMPLES \
  --seed=27\
  --train_batch_size=10 \
  --accumulate_grad_batches=10 \
  --val_batch_size=10 \
  --max_seq_length=512 \
  --num_workers=4 \
  --gpus=1 \
  --optimizer=AdamW \
  --lr=3e-3 \
  --weight_decay=5e-5 \
  --scheduler=StepLR \
  --t_0=2 \
  --t_mult=2 \
  --gamma=1.0 \
  --step_size=1000 \
  --max_epochs=100 \
  --precision=32 \
  --gradient_clip_val=1.0 \
  --check_val_every_n_epoch=3 \


args Namespace(accelerator=None, accumulate_grad_batches=10, amp_backend='native', amp_level=None, auto_lr_find=False, auto_scale_batch_size=False, auto_select_gpus=False, benchmark=False, check_val_every_n_epoch=3, checkpoint_callback=None, default_root_dir=None, detect_anomaly=False, deterministic=False, devices=None, enable_checkpointing=True, enable_model_summary=True, enable_progress_bar=True, fast_dev_run=False, flush_logs_every_n_steps=None, gamma=1.0, gpus=1, gradient_clip_algorithm=None, gradient_clip_val=1.0, input_test='/content/drive/MyDrive/MESTRADO/projeto/Experimentos/SCAN/Verificador/generate-train-examples/test_hipoteses.tsv', input_train='/content/drive/MyDrive/MESTRADO/projeto/Experimentos/SCAN/Verificador/generate-train-examples/train_hipoteses.tsv', input_val='/content/drive/MyDrive/MESTRADO/projeto/Experimentos/SCAN/Verificador/generate-train-examples/test_hipoteses.tsv', ipus=None, limit_predict_batches=1.0, limit_test_batches=1.0, limit_train_batches=1.0, limit_

In [ ]:
#https://app.neptune.ai/mirelle/scan/e/SCAN-163